# Megaleiloes Data Scraper
#### Made By- Uday S.
#### For- Raphael G.

## I. Importing Libraries

In [1]:
#importing drivers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
#importing beautiful soup
from bs4 import BeautifulSoup
#importing the necessary libraries
import pandas as pd, numpy as np
#importing json package to read the data
import json
#importing os module
import os
from os.path import basename
#importing time
import time
#importing pathlib
import pathlib
#importing string
import string

## II. Defining Variable

In [2]:
#defining a function to find the position of a substring's nth instance in a string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [5]:
#Defining xpaths
xpath_name= "/html/body/div[3]/div[2]/div[2]/div[2]/div/h1"
xpath_name_alt= "/html/body/div[3]/div[3]/div[2]/div[2]/div/h1"
xpath_main= "/html/body/div[3]/div[3]/div[2]/div[2]/div"
xpath_main_alt= "/html/body/div[3]/div[2]/div[2]/div[2]/div"

## III. Scraping Page Links

In [3]:
# Using Selenium to define driver
driver = webdriver.Chrome(ChromeDriverManager().install())
#defining the base url
url= "https://www.megaleiloes.com.br/imoveis"
#opening the url
driver.get(url)
#defining the variable to be used in the for loop
p_db=""
#iterating through the pages using a for loop to get the links of all the listings
for i in range(13):
    time.sleep(5) #waiting for the webpage to load
    p_element = driver.find_element_by_xpath('//*[@id="w0"]/div[1]') #getting the required element
    p_db= p_db + " " +p_element.get_attribute('innerHTML')# Getting attributes of an element
    #using try & except so that an error doesn't come up if pages are less than 13
    try:
        click_element = driver.find_element_by_xpath('//*[@id="w0"]/div[2]/ul/li[11]/a')
        click_element.click()
    except:
        pass
#Creating an empty DataFrame
url_df= pd.DataFrame()
#Creating an empty "url" column with the required rows
url_df["url"]= [None] * 584
#Using a for loop to add the listing's link to the appropriate row in the "url" column
for i in range(584):
    try:
        url_df["url"][i]= p_db[find_nth(p_db,'class="card-title" href="',i+1)+25:find_nth(p_db,'" data-pjax="0">',(3*i)+1)] #Using .find() to index the strings in order to get the link in the correct form
        a=i
    except:
        print(i)

## IV. Pages' Data Scraping

### IV. A) Defining Df Columns

In [4]:
#Defining empty columns
url_df["Name"]=""
url_df["Location"]="" 
url_df["1st Square Date"]=""
url_df["Evaluation Value 1ª"]=""
url_df["2nd Square Date"]=""
url_df["Valuation Value 2nd"]=""
url_df["Visits"]=""
url_df["Description"]=""
url_df["Purchase and sale contract"]=""
url_df["Vara"]=""
url_df["Forum"]=""
url_df["Auctioneer"]=""
url_df["Author"]=""
url_df["Defendant"]=""
url_df["Process number"]=""
url_df["Site Process"]=""

### IV. B) Scraping Data Onto Respective Df Columns

In [6]:
# Using Selenium to define driver
driver = webdriver.Chrome(ChromeDriverManager().install())
#Using for loop to iterate through the links
for i in range(584):
    driver.get(url_df["url"][i]) #opening the link
    time.sleep(1) #waiting for the webpage to load
    #Using try/except to skip any exception due to different data structure for name_element
    try:
        name_element= driver.find_element_by_xpath(xpath_name)
    except:
        name_element= driver.find_element_by_xpath(xpath_name_alt)
    #Using try/except to skip any exception due to missing second instance date & value
    try:
        seconddate_element= driver.find_elements_by_class_name('card-second-instance-date')[0]
        firstdate_element= driver.find_elements_by_class_name('card-first-instance-date')[0]
        firstprice_element= driver.find_elements_by_class_name('card-instance-value')[0] 
        secondprice_element= driver.find_elements_by_class_name('card-instance-value')[1]
    except:
        firstdate_element= driver.find_elements_by_class_name('card-first-instance-date')[1]
        firstprice_element= driver.find_elements_by_class_name('card-instance-value')[0]
        seconddate_element=""
        secondprice_element=""
    visits_element= driver.find_element_by_class_name("views-count")
    desc_element= driver.find_element_by_xpath('//*[@id="tab-description"]/div')
    contract_element= driver.find_element_by_xpath('//*[@id="tab-contract"]/div')
    #Using try/except to skip any exception due to different data structure for the main data description elements
    try:
        vara_element= driver.find_element_by_xpath(xpath_main)
        forum_element= driver.find_element_by_xpath(xpath_main)
        leiloeiro_element= driver.find_element_by_xpath(xpath_main)
        author_element= driver.find_element_by_xpath(xpath_main)
        def_element= driver.find_element_by_xpath(xpath_main)
        num_element= driver.find_element_by_xpath(xpath_main)
        site_element= driver.find_element_by_xpath(xpath_main)
        loc_element= driver.find_element_by_xpath(xpath_main)
    except:
        vara_element= driver.find_element_by_xpath(xpath_main_alt)
        forum_element= driver.find_element_by_xpath(xpath_main_alt)
        leiloeiro_element= driver.find_element_by_xpath(xpath_main_alt)
        author_element= driver.find_element_by_xpath(xpath_main_alt)
        def_element= driver.find_element_by_xpath(xpath_main_alt)
        num_element= driver.find_element_by_xpath(xpath_main_alt)
        site_element= driver.find_element_by_xpath(xpath_main_alt)
        loc_element= driver.find_element_by_xpath(xpath_main_alt)
    #Getting the attribute/text from the elements defined above
    url_df["Name"][i]=name_element.get_attribute('innerHTML')
    url_df["1st Square Date"][i]= firstdate_element.text
    url_df["Evaluation Value 1ª"][i]= firstprice_element.text
    try:
        url_df["2nd Square Date"][i]= seconddate_element.text
        url_df["Valuation Value 2nd"][i]= secondprice_element.text
    except:
        pass
    url_df["Location"][i]= loc_element.get_attribute('innerHTML')
    url_df["Visits"][i]= visits_element.get_attribute('innerHTML')
    url_df["Description"][i]= desc_element.get_attribute('innerHTML')
    url_df["Purchase and sale contract"][i]= contract_element.get_attribute('innerHTML')
    url_df["Vara"][i]= vara_element.get_attribute('innerHTML')
    url_df["Forum"][i]= forum_element.get_attribute('innerHTML')
    url_df["Auctioneer"][i]= leiloeiro_element.get_attribute('innerHTML')
    url_df["Author"][i]= author_element.get_attribute('innerHTML')
    url_df["Defendant"][i]= def_element.get_attribute('innerHTML')
    url_df["Process number"][i]= num_element.get_attribute('innerHTML')
    url_df["Site Process"][i]= site_element.get_attribute('innerHTML')

### V. Cleaning Data in the Df

In [7]:
#Using a for loop to iterate through all the rows in the dataframe
for i in range(584):
    #cleaning the Location data using find_nth function defined above
    url_df["Location"][i]= url_df["Location"][i][find_nth(url_df["Location"][i],'Localização',1)+65:find_nth(url_df["Location"][i][find_nth(url_df["Location"][i],'Localização',1)+65:],'</div',1)+find_nth(url_df["Location"][i],'Localização',1)+65]
    #cleaning the Location data using replace() & strip()
    url_df["1st Square Date"][i]=url_df["1st Square Date"][i].replace("1ª Praça: ","").strip("Data: ") 
    #cleaning the data using replace()
    url_df["2nd Square Date"][i]=url_df["2nd Square Date"][i].replace("2ª Praça: ","") 
    #cleaning the Location data using find_nth function defined above
    url_df["Visits"][i]=url_df["Visits"][i][find_nth(url_df["Visits"][i],'"value">',1)+8:][:find_nth(url_df["Visits"][i][find_nth(url_df["Visits"][i],'"value">',1)+8:],'</',1)]
    #cleaning the Location data using find_nth function defined above, and strip()
    url_df["Description"][i]=url_df["Description"][i].strip('\n        ')[:find_nth(url_df["Description"][i].strip('\n        '),'<b',1)]
    #cleaning the Location data using strip() & replace()
    url_df["Purchase and sale contract"][i]=url_df["Purchase and sale contract"][i].replace("<br>","").replace("\n","").strip(" ")
    #cleaning the Location data using find_nth function defined above, and find()
    url_df["Vara"][i]=url_df["Vara"][i][url_df["Vara"][i].find("Vara"):][url_df["Vara"][i][url_df["Vara"][i].find("Vara"):].find("value")+7:find_nth(url_df["Vara"][i][url_df["Vara"][i].find("Vara"):],"</d",2)]
    #cleaning the Location data using find_nth function defined above, and find()
    url_df["Forum"][i]=url_df["Forum"][i][url_df["Forum"][i].find("Forum"):][url_df["Forum"][i][url_df["Forum"][i].find("Forum"):].find("value")+7:find_nth(url_df["Forum"][i][url_df["Forum"][i].find("Forum"):],"</d",2)]
    #cleaning the Location data using find_nth function defined above, find(), and strip()
    url_df["Auctioneer"][i]=url_df["Auctioneer"][i][url_df["Auctioneer"][i].find("Leiloeiro"):][url_df["Auctioneer"][i][url_df["Auctioneer"][i].find("Leiloeiro"):].find("value")+7:][:url_df["Auctioneer"][i][url_df["Auctioneer"][i].find("Leiloeiro"):][url_df["Auctioneer"][i][url_df["Auctioneer"][i].find("Leiloeiro"):].find("value")+7:].find("</")].strip("\n").strip(" ").strip("\n")
    #cleaning the Location data using find_nth function defined above, and find()
    url_df["Author"][i]=url_df["Author"][i][url_df["Author"][i].find("Autor"):][url_df["Author"][i][url_df["Author"][i].find("Autor"):].find("value")+7:find_nth(url_df["Author"][i][url_df["Author"][i].find("Autor"):],"</d",2)]
    #cleaning the Location data using find_nth function defined above, and find()
    url_df["Defendant"][i]=url_df["Defendant"][i][url_df["Defendant"][i].find("Réu"):][url_df["Defendant"][i][url_df["Defendant"][i].find("Réu"):].find("value")+7:find_nth(url_df["Defendant"][i][url_df["Defendant"][i].find("Réu"):],"</d",2)]
    #cleaning the Location data using find_nth function defined above, and find()
    url_df["Site Process"][i]=url_df["Site Process"][i][url_df["Site Process"][i].find("https://esaj"):][:url_df["Site Process"][i][url_df["Site Process"][i].find("https://esaj"):].find('"')]
    #using if-else statement to assign the correct value to the "Process number" column based on "Site Process" column's value
    if url_df["Site Process"][i] == "":
        url_df["Process number"][i]= ""
    else:
        url_df["Process number"][i]=url_df["Process number"][i][url_df["Process number"][i].find('"top"')+6:][:url_df["Process number"][i][url_df["Process number"][i].find('"top"')+6:].find(" ()")]

### VI. Downloading The Data

In [8]:
#Downloading the url_df to an excel file
url_df.to_excel("Mgaleiloes.xlsx",index=False)